# Schedule Access Plan Recommendation

This notebook is dedicated to model fitting in terms of database access plans.

In [15]:
# pandas
import pandas as pd
print('pandas: %s' % pd.__version__)
# numpy
import numpy as np
print('numpy: %s' % np.__version__)
# sklearn
import sklearn as sk
from sklearn import preprocessing
print('sklearn: %s' % sk.__version__)

pandas: 0.23.4
numpy: 1.15.2
sklearn: 0.18.1


In [16]:
#
# Experiment Config
tpcds='TPCDS1' # Schema upon which to operate test

### Read data from file into pandas dataframes

In [17]:
rep_hist_snapshot_path = 'C:/Users/gabriel.sammut/University/Data_ICS5200/Schedule/' + tpcds + '/v2/rep_hist_snapshot.csv'
rep_vsql_plan_path = 'C:/Users/gabriel.sammut/University/Data_ICS5200/Schedule/' + tpcds + '/v2/rep_vsql_plan.csv'
# rep_hist_snapshot_path = 'D:/Projects/Datagenerated_ICS5200/Schedule/' + tpcds + '/rep_hist_snapshot.csv'
# rep_vsql_plan_path = 'D:/Projects/Datagenerated_ICS5200/Schedule/' + tpcds + '/rep_vsql_plan.csv'
#
rep_hist_snapshot_df = pd.read_csv(rep_hist_snapshot_path)
rep_vsql_plan_df = pd.read_csv(rep_vsql_plan_path)
#
def prettify_header(headers):
    """
    Cleans header list from unwated character strings
    """
    header_list = []
    [header_list.append(header.replace("(","").replace(")","").replace("'","").replace(",","")) for header in headers]
    return header_list
#
rep_hist_snapshot_df.columns = prettify_header(rep_hist_snapshot_df.columns.values)
rep_vsql_plan_df.columns = prettify_header(rep_vsql_plan_df.columns.values)
print('------------------------------------------')
print(rep_hist_snapshot_df.columns)
print(rep_vsql_plan_df.columns)

------------------------------------------
Index(['SNAP_ID', 'DBID', 'INSTANCE_NUMBER', 'SQL_ID', 'PLAN_HASH_VALUE',
       'OPTIMIZER_COST', 'OPTIMIZER_MODE', 'OPTIMIZER_ENV_HASH_VALUE',
       'SHARABLE_MEM', 'LOADED_VERSIONS', 'VERSION_COUNT', 'MODULE', 'ACTION',
       'SQL_PROFILE', 'FORCE_MATCHING_SIGNATURE', 'PARSING_SCHEMA_ID',
       'PARSING_SCHEMA_NAME', 'PARSING_USER_ID', 'FETCHES_TOTAL',
       'FETCHES_DELTA', 'END_OF_FETCH_COUNT_TOTAL', 'END_OF_FETCH_COUNT_DELTA',
       'SORTS_TOTAL', 'SORTS_DELTA', 'EXECUTIONS_TOTAL', 'EXECUTIONS_DELTA',
       'PX_SERVERS_EXECS_TOTAL', 'PX_SERVERS_EXECS_DELTA', 'LOADS_TOTAL',
       'LOADS_DELTA', 'INVALIDATIONS_TOTAL', 'INVALIDATIONS_DELTA',
       'PARSE_CALLS_TOTAL', 'PARSE_CALLS_DELTA', 'DISK_READS_TOTAL',
       'DISK_READS_DELTA', 'BUFFER_GETS_TOTAL', 'BUFFER_GETS_DELTA',
       'ROWS_PROCESSED_TOTAL', 'ROWS_PROCESSED_DELTA', 'CPU_TIME_TOTAL',
       'CPU_TIME_DELTA', 'ELAPSED_TIME_TOTAL', 'ELAPSED_TIME_DELTA',
       'IOWAIT_TO

### Dealing with empty values

In [18]:
def get_na_columns(df, headers):
    """
    Return columns which consist of NAN values
    """
    na_list = []
    for head in headers:
        if df[head].isnull().values.any():
            na_list.append(head)
    return na_list
#
print('N/A Columns\n')
print('\nREP_HIST_SNAPSHOT Features ' + str(len(rep_hist_snapshot_df.columns)) + ': ' + str(get_na_columns(df=rep_hist_snapshot_df,headers=rep_hist_snapshot_df.columns)) + "\n")
print('\nREP_VSQL_PLAN Features ' + str(len(rep_vsql_plan_df.columns)) + ': ' + str(get_na_columns(df=rep_vsql_plan_df,headers=rep_vsql_plan_df.columns)) + "\n")
#
def fill_na(df):
    """
    Replaces NA columns with 0s
    """
    return df.fillna(0)
#
# Populating NaN values with amount '0'
rep_hist_snapshot_df = fill_na(df=rep_hist_snapshot_df)
rep_vsql_plan_df = fill_na(df=rep_vsql_plan_df)

N/A Columns


REP_HIST_SNAPSHOT Features 90: ['OPTIMIZER_COST', 'MODULE', 'ACTION', 'SQL_PROFILE', 'BIND_DATA', 'FLAG']


REP_VSQL_PLAN Features 39: ['OPTIONS', 'OBJECT_NODE', 'OBJECT#', 'OBJECT_OWNER', 'OBJECT_NAME', 'OBJECT_ALIAS', 'OBJECT_TYPE', 'OPTIMIZER', 'PARENT_ID', 'COST', 'CARDINALITY', 'BYTES', 'OTHER_TAG', 'PARTITION_START', 'PARTITION_STOP', 'PARTITION_ID', 'OTHER', 'DISTRIBUTION', 'CPU_COST', 'IO_COST', 'TEMP_SPACE', 'ACCESS_PREDICATES', 'FILTER_PREDICATES', 'PROJECTION', 'TIME', 'QBLOCK_NAME', 'REMARKS', 'OTHER_XML']



### TPC-DS Filtering & Merging

Exclude all queries but those pertaining to TPC-DS.
This part also merges the field 'SQL_TEXT' from data matrix REP_HIST_SNAPSHOT with data matrix 'REP_VSQL_PLAN'.

In [19]:
# #
# # Segregating those SQL_IDs which utilize TPC-DS owned objects
# # rep_vsql_plan_series = rep_vsql_plan_df['SQL_ID'].loc[rep_vsql_plan_df['OBJECT_OWNER'] == tpcds]
# rep_vsql_plan_series = rep_vsql_plan_df['SQL_ID'].loc[rep_vsql_plan_df['OBJECT_OWNER'].str.contains([tpcds,'SYS'])]
# tpc_sql_ids = rep_vsql_plan_series.unique()
# print('Total SQL_IDs [' + str(len(tpc_sql_ids)) + ']')
# print('Unique ' + tpcds + ' IDs: ' + str(tpc_sql_ids))
# #
# # Retrieves data matrix rows pertaining to SQL_IDs using TPC-DS owned objects
# print("\nREP_HIST_SNAPSHOT Shape before: " + str(rep_hist_snapshot_df.shape))
# rep_hist_snapshot_df = rep_hist_snapshot_df.loc[rep_hist_snapshot_df['SQL_ID'].isin(tpc_sql_ids)]
# print("REP_HIST_SNAPSHOT after: " + str(rep_hist_snapshot_df.shape))
# #
# print("\nREP_VSQL_PLAN Shape before: " + str(rep_vsql_plan_df.shape))
# rep_vsql_plan_df = rep_vsql_plan_df.loc[rep_vsql_plan_df['SQL_ID'].isin(tpc_sql_ids)]
# print("REP_VSQL_PLAN Shape after: " + str(rep_vsql_plan_df.shape))
# #
# # Group By Values by SNAP_ID, PLAN_HASH_VALUE , sum all metrics (for matrix REP_HIST_SNAPSHOT)
# rep_hist_snapshot_df = rep_hist_snapshot_df.groupby(['SQL_TEXT','SQL_ID','PLAN_HASH_VALUE']).sum()
# rep_hist_snapshot_df.reset_index(inplace=True)
# #
# # Merging data matrixes
# df = pd.merge(rep_hist_snapshot_df, rep_vsql_plan_df, how='inner', on=['SQL_ID','PLAN_HASH_VALUE','DBID'])
# #
# # Dropping / Renaming transformed columns as a result of merge
# df.drop(columns=['CON_DBID_x','CON_ID_x'],inplace=True)
# df.rename(index=str, columns={"SQL_ID_y": "SQL_ID", 
#                               "DBID_y": "DBID", 
#                               "CON_DBID_y":"CON_DBID",
#                               "CON_ID_y":"CON_ID"},inplace=True)
# print("\nMerged Dataframe: " + str(df.shape))
# print("Columns: ")
# print(df.columns.values)
# print(df.head(100))
# #print(df[['SQL_TEXT','OPERATION','OPTIONS','ID']].head(20))

In [20]:
#
# Creating a dataframe consisting of a mapping library, composed of SQL_IDs & SQL_TEXTs
sql_id_map = rep_hist_snapshot_df[['SQL_ID','SQL_TEXT']]
print(sql_id_map.shape)
#
# Drop duplicate combinations from map
sql_id_map.drop_duplicates(keep = False,
                           inplace = True)
print(sql_id_map.shape)
#
# Merge SQL_TEXT with Access Plan table
df = pd.merge(sql_id_map, rep_vsql_plan_df, how='inner', on=['SQL_ID'])
print('Plan table shape: ' + str(rep_vsql_plan_df.shape))
print("Merged matrix shape: " + str(df.shape) + "\n\n--------------------------------------------------------\n\n")
print(df.head())

(46485, 2)
(253, 2)
Plan table shape: (2712, 39)
Merged matrix shape: (1280, 40)

--------------------------------------------------------


          SQL_ID                                           SQL_TEXT  \
0  056v28tcwy8ms  /* SQL Analyze(0) */ select /*+  full(t)    pa...   
1  056v28tcwy8ms  /* SQL Analyze(0) */ select /*+  full(t)    pa...   
2  056v28tcwy8ms  /* SQL Analyze(0) */ select /*+  full(t)    pa...   
3  056v28tcwy8ms  /* SQL Analyze(0) */ select /*+  full(t)    pa...   
4  056v28tcwy8ms  /* SQL Analyze(0) */ select /*+  full(t)    pa...   

         DBID  PLAN_HASH_VALUE  ID         OPERATION      OPTIONS OBJECT_NODE  \
0  2634225673       3274964688   0  SELECT STATEMENT            0           0   
1  2634225673       3274964688   1              SORT    AGGREGATE           0   
2  2634225673       3274964688   2    PX COORDINATOR            0           0   
3  2634225673       3274964688   3           PX SEND  QC (RANDOM)      :Q1000   
4  2634225673       3274964

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


### Feature Selection

In this step, redundant features are dropped. Features are considered redundant if exhibit a standard devaition of 0 (meaning no change in value).

In [21]:
def drop_flatline_columns(df):
    columns = df.columns
    flatline_features = []
    for i in range(len(columns)):
        try:
            std = df[columns[i]].std()
            if std == 0:
                flatline_features.append(columns[i])
        except:
            pass
    #
    #print('Features which are considered flatline:\n')
    #for col in flatline_features:
    #    print(col)
    print('\nShape before changes: [' + str(df.shape) + ']')
    df = df.drop(columns=flatline_features)
    print('Shape after changes: [' + str(df.shape) + ']')
    print('Dropped a total [' + str(len(flatline_features)) + ']')
    return df
#
df = drop_flatline_columns(df=df)
print('\nAfter flatline column drop:')
print(df.shape)
print(df.columns)


Shape before changes: [(1280, 40)]
Shape after changes: [(1280, 32)]
Dropped a total [8]

After flatline column drop:
(1280, 32)
Index(['SQL_ID', 'SQL_TEXT', 'PLAN_HASH_VALUE', 'ID', 'OPERATION', 'OPTIONS',
       'OBJECT_NODE', 'OBJECT#', 'OBJECT_OWNER', 'OBJECT_NAME', 'OBJECT_ALIAS',
       'OBJECT_TYPE', 'OPTIMIZER', 'PARENT_ID', 'DEPTH', 'POSITION',
       'SEARCH_COLUMNS', 'COST', 'CARDINALITY', 'BYTES', 'OTHER_TAG',
       'PARTITION_START', 'PARTITION_STOP', 'PARTITION_ID', 'DISTRIBUTION',
       'CPU_COST', 'IO_COST', 'TEMP_SPACE', 'TIME', 'QBLOCK_NAME', 'TIMESTAMP',
       'OTHER_XML'],
      dtype='object')


### Data Ordering

Sorting of datasets in order of 
* SNAP_ID
* SQL_ID
* ID

In [22]:
df.sort_values(by=['TIMESTAMP','SQL_ID','ID'], ascending=True, inplace=True)
print(df.head())

          SQL_ID                                           SQL_TEXT  \
0  056v28tcwy8ms  /* SQL Analyze(0) */ select /*+  full(t)    pa...   
1  056v28tcwy8ms  /* SQL Analyze(0) */ select /*+  full(t)    pa...   
2  056v28tcwy8ms  /* SQL Analyze(0) */ select /*+  full(t)    pa...   
3  056v28tcwy8ms  /* SQL Analyze(0) */ select /*+  full(t)    pa...   
4  056v28tcwy8ms  /* SQL Analyze(0) */ select /*+  full(t)    pa...   

   PLAN_HASH_VALUE  ID         OPERATION      OPTIONS OBJECT_NODE  OBJECT#  \
0       3274964688   0  SELECT STATEMENT            0           0      0.0   
1       3274964688   1              SORT    AGGREGATE           0      0.0   
2       3274964688   2    PX COORDINATOR            0           0      0.0   
3       3274964688   3           PX SEND  QC (RANDOM)      :Q1000      0.0   
4       3274964688   4              SORT    AGGREGATE      :Q1000      0.0   

  OBJECT_OWNER OBJECT_NAME                        ...                          \
0            0         

### Label Encoding

Converting labels/features into numerical representations

In [23]:
def encode(df, encoded_labels):
    for col in df.columns:
        if col in encoded_labels:
            le = preprocessing.LabelEncoder()
            df[col] = le.fit_transform(df[col].astype(str))
    return df
#
#print(rep_vsql_plan_df.head())
encoded_labels = ['OPERATION','OPTIONS','OBJECT_OWNER','OBJECT_NAME','OBJECT_ALIAS','OBJECT_TYPE','OPTIMIZER','OTHER_TAG','DISTRIBUTION','QBLOCK_NAME']
df = encode(df=df, encoded_labels=encoded_labels)
print('Encoded labels:\n' + str(encoded_labels) + "\n\n----------------------------------------------\n\n")
print(df.head())

Encoded labels:
['OPERATION', 'OPTIONS', 'OBJECT_OWNER', 'OBJECT_NAME', 'OBJECT_ALIAS', 'OBJECT_TYPE', 'OPTIMIZER', 'OTHER_TAG', 'DISTRIBUTION', 'QBLOCK_NAME']

----------------------------------------------


          SQL_ID                                           SQL_TEXT  \
0  056v28tcwy8ms  /* SQL Analyze(0) */ select /*+  full(t)    pa...   
1  056v28tcwy8ms  /* SQL Analyze(0) */ select /*+  full(t)    pa...   
2  056v28tcwy8ms  /* SQL Analyze(0) */ select /*+  full(t)    pa...   
3  056v28tcwy8ms  /* SQL Analyze(0) */ select /*+  full(t)    pa...   
4  056v28tcwy8ms  /* SQL Analyze(0) */ select /*+  full(t)    pa...   

   PLAN_HASH_VALUE  ID  OPERATION  OPTIONS OBJECT_NODE  OBJECT#  OBJECT_OWNER  \
0       3274964688   0         20        0           0      0.0             0   
1       3274964688   1         21        1           0      0.0             0   
2       3274964688   2         17        0           0      0.0             0   
3       3274964688   3         18      

### TPC_DS Queries Under Test

Displaying (TPC-DS) queries filtered for upcoming use.

In [24]:
#
# Query Distribution Test
print(df)

             SQL_ID                                           SQL_TEXT  \
0     056v28tcwy8ms  /* SQL Analyze(0) */ select /*+  full(t)    pa...   
1     056v28tcwy8ms  /* SQL Analyze(0) */ select /*+  full(t)    pa...   
2     056v28tcwy8ms  /* SQL Analyze(0) */ select /*+  full(t)    pa...   
3     056v28tcwy8ms  /* SQL Analyze(0) */ select /*+  full(t)    pa...   
4     056v28tcwy8ms  /* SQL Analyze(0) */ select /*+  full(t)    pa...   
5     056v28tcwy8ms  /* SQL Analyze(0) */ select /*+  full(t)    pa...   
6     056v28tcwy8ms  /* SQL Analyze(0) */ select /*+  full(t)    pa...   
7     056v28tcwy8ms  /* SQL Analyze(0) */ select /*+  full(t)    pa...   
46    c7n4sw1c9nh64  SELECT /* DS_SVC */ /*+ dynamic_sampling(0) no...   
47    c7n4sw1c9nh64  SELECT /* DS_SVC */ /*+ dynamic_sampling(0) no...   
48    c7n4sw1c9nh64  SELECT /* DS_SVC */ /*+ dynamic_sampling(0) no...   
49    c7n4sw1c9nh64  SELECT /* DS_SVC */ /*+ dynamic_sampling(0) no...   
50    c7n4sw1c9nh64  SELECT /* DS_SVC 

### Access Plan Resource Aggregation

This method attempts to tackle the problem of access plan anomolies by aggregating resources per explain plan. Notable resources which are being considered are as follows:

* COST
* CARDINALITY
* BYTES
* PARTITION_DELTA (Partition End - Partition Start)
* CPU_COST
* IO_COST
* TEMP_SPACE
* TIME

The reasoning behind these fields in particular is mainly because these columns can be aggregated together.

In [26]:
df_aggregate = df.groupby(['SQL_TEXT','PLAN_HASH_VALUE']).sum()
df_aggregate.reset_index(inplace=True)
print(df_aggregate.columns)
print(df_aggregate.shape)
for index, row in df_aggregate.iterrows():
    print(row['SQL_TEXT'])

Index(['SQL_TEXT', 'PLAN_HASH_VALUE', 'ID', 'OPERATION', 'OPTIONS', 'OBJECT#',
       'OBJECT_OWNER', 'OBJECT_NAME', 'OBJECT_ALIAS', 'OBJECT_TYPE',
       'OPTIMIZER', 'PARENT_ID', 'DEPTH', 'POSITION', 'SEARCH_COLUMNS', 'COST',
       'CARDINALITY', 'BYTES', 'OTHER_TAG', 'PARTITION_ID', 'DISTRIBUTION',
       'CPU_COST', 'IO_COST', 'TEMP_SPACE', 'TIME', 'QBLOCK_NAME'],
      dtype='object')
(84, 26)
/* SQL Analyze(0) */ select /*+  full(t)    parallel(t,60) parallel_index(t,60) dbms_stats cursor_sharing_exact use_weak_name_resl dynamic_sampling(0) no_monitoring xmlindex_sel_idx_tbl no_substrb_pad  */to_char(count("SR_STORE_SK")),substrb(dump(min("SR_STORE_SK"),16,0,64),1,240),substrb(dump(max("SR_STORE_SK"),16,0,64),1,240),to_char(count("SR_REASON_SK")),substrb(dump(min("SR_REASON_SK"),16,0,64),1,240),substrb(dump(max("SR_REASON_SK"),16,0,64),1,240),to_char(count("SR_TICKET_NUMBER")),substrb(dump(min("SR_TICKET_NUMBER"),16,0,64),1,240),substrb(dump(max("SR_TICKET_NUMBER"),16,0,64),1,24